In [ ]:
#r "nuget:FParsec"
#r "nuget:Expecto"

Installing Packages Expecto FParsec

Installing Packages Expecto. FParsec.

Installing Packages Expecto.. FParsec..

Installing Packages Expecto... FParsec...

Installing Packages Expecto.... FParsec....

Installing Packages Expecto..... FParsec.....

Installing Packages Expecto...... FParsec......

Installing Packages Expecto....... FParsec.......

Installed Packages Expecto, 10.1.0 FParsec, 1.1.1

In [ ]:
open FParsec


type Block =
    {
        magic : string
        content : string
    }

let magicMarker = pstring "#!"

let magicCommand =
    magicMarker
    >>. manyTill anyChar newline
    |>> (String.Concat >> fun s -> s.Trim ())

let content =
    (newline >>. magicMarker) <|> (eof >>. preturn "")
    |> attempt
    |> lookAhead
    |> manyTill anyChar
    |>> (String.Concat >> fun s -> s.Trim ())

let block =
    pipe2
        magicCommand
        content
        (fun magic content ->
            {
                magic = magic
                content = content
            })

let blocks =
    skipMany newline
    >>. sepEndBy block (skipMany1 newline)


let formatBlock kernel (block : Block) =
    match kernel, block with
    | _, { magic = "markdown"; content = content } ->
        content.Split [| '\n' |]
        |> Array.map (fun line -> $"// {line.Trim ()}")
        |> String.concat "\n"
    | "fsharp", { magic = "fsharp"; content = content } when 
        (content.StartsWith "//// test"
        || content.StartsWith "//// ignore")
        |> not ->
            content.Split [| '\n' |]
            |> Array.filter (fun line -> line.StartsWith "#r" |> not)
            |> String.concat "\n"
    | _ -> ""

let formatBlocks kernel blocks =
    blocks
    |> List.map (formatBlock kernel)
    |> List.filter (fun s -> s <> "")
    |> String.concat "\n\n"

let run input =
    match run blocks input with
    | Success (result, _, _) -> Result.Ok result
    | Failure (errorMsg, _, _) -> Result.Error errorMsg

let parseDibCode kernel file =
    let input = File.ReadAllText file
    match run input with
    | Result.Ok blocks ->
        blocks
        |> List.filter (fun block -> block.magic = kernel || block.magic = "markdown")
        |> formatBlocks kernel
    | Result.Error msg -> failwith msg

let writeDibCode kernel file =
    printfn $"Parsing {file}"
    let output = parseDibCode kernel file
    let outputFileName =
        match kernel with
        | "fsharp" -> file.Replace (".dib", ".fs")
        | _ -> failwith "Unknown kernel"
    File.WriteAllText (outputFileName, output)

In [ ]:
let escapeCell (input : string) =
    input.Split [| '\n' |]
    |> Array.map (fun line ->
        if line.StartsWith "\\#!"
        then System.Text.RegularExpressions.Regex.Replace (line, "^\\\\#!", "#!")
        else line)
    |> String.concat "\n"

In [ ]:
//// ignore

let _equal expected actual =
    "" |> Expecto.Expect.equal actual expected
    printfn $"{actual}"

In [ ]:
//// ignore

let example1 =
    """#!meta

{"kernelInfo":{"defaultKernelName":"csharp","items":[{"aliases":[],"name":"csharp"},{"aliases":[],"name":"fsharp"}]}}

\#!fsharp

##r "nuget:Expecto"

\#!markdown

## ParserLibrary

\#!fsharp

open System

\#!markdown

### TextInput

\#!fsharp

type Position =
    {
        line : int
        column : int
    }"""
    |> escapeCell

printfn $"{example1}"

let blocks =
    run example1
    |> Result.toOption
    |> Option.get
blocks

index value 0 { magic = "meta"\n content =\n "{"kernelInfo":{"defaultKernelName":"csharp","items":[{"aliases":[],"name":"csharp"},{"aliases":[],"name":"fsharp"}]}}" } magic meta content {"kernelInfo":{"defaultKernelName":"csharp","items":[{"aliases":[],"name":"csharp"},{"aliases":[],"name":"fsharp"}]}} 1 { magic = "fsharp"\n content = "##r "nuget:Expecto"" } magic fsharp content ##r "nuget:Expecto" 2 { magic = "markdown"\n content = "## ParserLibrary" } magic markdown content ## ParserLibrary 3 { magic = "fsharp"\n content = "open System" } magic fsharp content open System 4 { magic = "markdown"\n content = "### TextInput" } magic markdown content ### TextInput 5 { magic = "fsharp"\n content =\n "type Position =\n {\n line : int\n column : int\n }" } magic fsharp content type Position =
 {
 line : int
 column : int
 }

#!meta

{"kernelInfo":{"defaultKernelName":"csharp","items":[{"aliases":[],"name":"csharp"},{"aliases":[],"name":"fsharp"}]}}

#!fsharp

##r "nuget:Expecto"

#!markdown

## ParserLibrary

#!fsharp

open System

#!markdown

### TextInput

#!fsharp

type Position =
    {
        line : int
        column : int
    }


In [ ]:
//// test

blocks
|> (formatBlocks "fsharp")
|> _equal """##r "nuget:Expecto"

// ## ParserLibrary

open System

// ### TextInput

type Position =
    {
        line : int
        column : int
    }"""

##r "nuget:Expecto"

// ## ParserLibrary

open System

// ### TextInput

type Position =
    {
        line : int
        column : int
    }


In [ ]:
//// test

blocks
|> (formatBlocks "markdown")
|> _equal """// ## ParserLibrary

// ### TextInput"""

// ## ParserLibrary

// ### TextInput


In [ ]:
//// ignore

match Environment.GetEnvironmentVariable "OUTPUT" with
| "" | null -> ()
| path when System.IO.File.Exists path -> path |> writeDibCode "fsharp"
| path when path.Contains ";" -> path.Split [| ';' |] |> Array.iter (writeDibCode "fsharp")
| _ -> System.IO.Path.Combine (System.IO.Directory.GetCurrentDirectory (), "DibParser.dib") |> writeDibCode "fsharp"

Parsing Parser.dib
Parsing JsonParser.dib
Parsing DibParser.dib
